In [ ]:
%%writefile teste2.c


In [ ]:
%%writefile teste2.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

typedef struct {
    int user_id;
    int item_id;
    float rating;
} Data;

// Função para carregar o CSV
Data* load_csv(const char* path, int* num_rows, char* header) {
    FILE* file = fopen(path, "r");
    if (!file) {
        fprintf(stderr, "Erro ao abrir o arquivo: %s\n", path);
        exit(EXIT_FAILURE);
    }

    printf("Arquivo aberto\n");

    int capacidade = 926055; // Número de linhas hardcoded, ajuste se necessário
    Data* data = malloc(capacidade * sizeof(Data));
    if (!data) {
        fprintf(stderr, "Falha na alocação de memória\n");
        fclose(file);
        exit(EXIT_FAILURE);
    }
    *num_rows = 0;

    printf("Memória alocada\n");

    // Remove a primeira linha (cabeçalho)
    if (fgets(header, 256, file) == NULL) {
        fprintf(stderr, "Erro ao ler a linha de cabeçalho\n");
        free(data);
        fclose(file);
        exit(EXIT_FAILURE);
    }
    printf("Cabeçalho lido: %s\n", header);

    while (fscanf(file, "%d,%d,%f\n", &data[*num_rows].user_id, &data[*num_rows].item_id, &data[*num_rows].rating) != EOF) {
        (*num_rows)++;
        if (*num_rows % 50 == 0) { // Imprime o progresso a cada 50 linhas
            printf("Linhas carregadas: %d\n", *num_rows);
        }
        if (*num_rows >= capacidade) {
            fprintf(stderr, "Capacidade pré-alocada excedida\n");
            break;
        }
        if (*num_rows <= 10) { // Imprime as primeiras linhas para depuração
            printf("Linha %d carregada: user_id=%d, item_id=%d, rating=%.2f\n", *num_rows, data[*num_rows-1].user_id, data[*num_rows-1].item_id, data[*num_rows-1].rating);
        }
    }

    printf("Dados carregados, total de linhas: %d\n", *num_rows);
    fclose(file);
    printf("Arquivo fechado\n");

    return data;
}

// Função para gerar rótulos
int* generate_labels(Data* data, int num_rows, int* num_labels, int use_user_id) {
    int* labels = malloc(num_rows * sizeof(int));
    int id = 0;

    for (int i = 0; i < num_rows; i++) {
        int value = use_user_id ? data[i].user_id : data[i].item_id;
        int found = 0;
        for (int j = 0; j < i; j++) {
            int compare_value = use_user_id ? data[j].user_id : data[j].item_id;
            if (value == compare_value) {
                labels[i] = labels[j];
                found = 1;
                break;
            }
        }
        if (!found) {
            labels[i] = id++;
        }
        if (i < 10) { // Imprime os primeiros rótulos para depuração
            printf("Linha %d: value=%d, label=%d\n", i, value, labels[i]);
        }
    }

    *num_labels = id;
    return labels;
}

// Função para exportar uma matriz para um arquivo
void export_matrix_to_file(float** matrix, int rows, int cols, const char* filename) {
    FILE* file = fopen(filename, "w");
    if (file == NULL) {
        fprintf(stderr, "Erro ao abrir o arquivo: %s\n", filename);
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            fprintf(file, "%f ", matrix[i][j]);
        }
        fprintf(file, "\n");
    }

    fclose(file);
    printf("Matriz exportada para %s\n", filename);
}

// Função para transpor uma matriz
void transpose(float** X, float** XT, int rows, int cols) {
    double start_time = omp_get_wtime();
    #pragma omp parallel for schedule(dynamic, 1)
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            XT[j][i] = X[i][j];
        }
        if ((i + 1) % 50 == 0) {
            #pragma omp critical
            {
                printf("Progresso da transposição: %d linhas completadas\n", i + 1);
            }
        }
    }
    double end_time = omp_get_wtime();
    printf("Transposição completada em %f segundos\n", end_time - start_time);
}

// Função para multiplicar duas matrizes
void matrix_multiply(float** A, float** B, float** C, int A_rows, int A_cols, int B_cols) {
    double start_time = omp_get_wtime();
    #pragma omp parallel for schedule(dynamic, 1)
    for (int i = 0; i < A_rows; i++) {
        for (int j = 0; j < B_cols; j++) {
            float sum = 0.0;
            for (int k = 0; k < A_cols; k++) {
                sum += A[i][k] * B[k][j];
            }
            C[i][j] = sum;
        }
        if ((i + 1) % 50 == 0) {
            #pragma omp critical
            {
                printf("Progresso da multiplicação: %d linhas completadas\n", i + 1);
            }
        }
    }
    double end_time = omp_get_wtime();
    printf("Multiplicação completada em %f segundos\n", end_time - start_time);
}

// Função para imprimir uma matriz (para depuração, não use se não necessário)
void print_matrix(float** matrix, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%f ", matrix[i][j]);
        }
        printf("\n");
    }
}

// Função para construir a matriz G
void build_g_matrix(float** G, Data* data, int num_rows, int num_user_labels, int num_item_labels, int* user_labels, int* item_labels, float reg) {
    // Aloca memória para matrizes temporárias
    float** X = malloc(num_item_labels * sizeof(float*));
    float** XT = malloc(num_user_labels * sizeof(float*));
    for (int i = 0; i < num_item_labels; i++) {
        X[i] = malloc(num_user_labels * sizeof(float));
    }
    for (int i = 0; i < num_user_labels; i++) {
        XT[i] = malloc(num_item_labels * sizeof(float));
    }

    // Inicializa a matriz X com zero
    #pragma omp parallel for
    for (int i = 0; i < num_item_labels; i++) {
        for (int j = 0; j < num_user_labels; j++) {
            X[i][j] = 0.0;
        }
    }

    // Constrói a matriz X usando IDs de usuários e itens rotulados
    #pragma omp parallel for
    for (int i = 0; i < num_rows; i++) {
        int user_label = user_labels[i];
        int item_label = item_labels[i];
        #pragma omp atomic
        X[item_label][user_label] += data[i].rating;
    }

    // Transpõe a matriz X
    transpose(X, XT, num_item_labels, num_user_labels);

    // Multiplica XT por X para obter G
    matrix_multiply(XT, X, G, num_user_labels, num_item_labels, num_user_labels);

    // Adiciona regularização à diagonal de G
    #pragma omp parallel for
    for (int i = 0; i < num_user_labels; i++) {
        G[i][i] += reg;
    }

    // Libera matrizes temporárias
    for (int i = 0; i < num_item_labels; i++) {
        free(X[i]);
    }
    for (int i = 0; i < num_user_labels; i++) {
        free(XT[i]);
    }
    free(X);
    free(XT);
}

// Função para inverter a matriz G
void invert_matrix(float** G, float** P, int num_labels) {
    // Para simplificação, assume-se que a matriz é invertível
    for (int i = 0; i < num_labels; i++) {
        for (int j = 0; j < num_labels; j++) {
            P[i][j] = (i == j) ? 1.0 / G[i][j] : -G[i][j] / (G[i][i] * G[j][j]);
        }
    }
}

// Função para construir a matriz B
void build_b_matrix(float** B, float** P, int num_labels) {
    for (int i = 0; i < num_labels; i++) {
        for (int j = 0; j < num_labels; j++) {
            B[i][j] = P[i][j] / (-P[i][i]);
        }
        B[i][i] += 1.0;
    }
}

int main() {
    // Código de inicialização...
    printf("Início\n");
    const char* path = "/content/trainBooksSmall.csv";
    int num_rows;

    // Aloca espaço para o cabeçalho
    char header[256];
    Data* data = load_csv(path, &num_rows, header);

    int num_user_labels, num_item_labels;
    int* user_labels = generate_labels(data, num_rows, &num_user_labels, 1);
    int* item_labels = generate_labels(data, num_rows, &num_item_labels, 0);

    float** G = malloc(num_user_labels * sizeof(float*));
    for (int i = 0; i < num_user_labels; i++) {
        G[i] = malloc(num_user_labels * sizeof(float));
    }

    float reg = 0.1;
    build_g_matrix(G, data, num_rows, num_user_labels, num_item_labels, user_labels, item_labels, reg);

    float** P = malloc(num_user_labels * sizeof(float*));
    for (int i = 0; i < num_user_labels; i++) {
        P[i] = malloc(num_user_labels * sizeof(float));
    }

    invert_matrix(G, P, num_user_labels);

    float** B = malloc(num_user_labels * sizeof(float*));
    for (int i = 0; i < num_user_labels; i++) {
        B[i] = malloc(num_user_labels * sizeof(float));
    }

    build_b_matrix(B, P, num_user_labels);

    export_matrix_to_file(G, num_user_labels, num_user_labels, "G_matrix.txt");
    export_matrix_to_file(P, num_user_labels, num_user_labels, "P_matrix.txt");
    export_matrix_to_file(B, num_user_labels, num_user_labels, "B_matrix.txt");

    // Libera memória
    for (int i = 0; i < num_user_labels; i++) {
        free(G[i]);
        free(P[i]);
        free(B[i]);
    }
    free(G);
    free(P);
    free(B);
    free(user_labels);
    free(item_labels);
    free(data);

    printf("Fim\n");
    return 0;
}

Writing teste2.c


In [ ]:
!gcc -o teste2 teste2.c -fopenmp

In [ ]:
!./teste2